In [ ]:
import ast
import pandas as pd
import pytholog as pl
import numpy as np

# Lectura de archivos

In [ ]:
df = pd.read_csv("/work/games_seeker/data/interim/games_requirements.csv", sep="|")

In [ ]:
df.loc[:, "Procesador"] = df.loc[:, "Procesador"].apply(lambda x: ast.literal_eval(x))
df.loc[:, "Gráficos"] = df.loc[:, "Gráficos"].apply(lambda x: ast.literal_eval(x))

In [ ]:
df

,name,url,Sistema operativo,Procesador,Memoria,Gráficos,Almacenamiento
0,Elden Ring,https://www.3djuegos.com/34802/elden-ring/,10,"[i5, r5]",12,[1060],60
1,Lost Ark,https://www.3djuegos.com/20774/lost-ark/,10,"[i3, r3]",8,[460],50
2,ELEX 2,https://www.3djuegos.com/39382/elex-2/,10,"[r3, i5]",12,[1060],45
3,Infernax,https://www.3djuegos.com/40542/infernax/,7,[i3],4,[320],1
4,Babylon's Fall,https://www.3djuegos.com/32561/babylons-fall/,10,"[i5, i5, r3, r5]",8,[1050],20
...,...,...,...,...,...,...,...
105,Battlezone: Gold Edition,https://www.3djuegos.com/32087/battlezone-gold...,7,[i5],8,[970],80
106,Bad Bots,https://www.3djuegos.com/16265/bad-bots/,7,"[i5, r5]",1,[1050],80
107,DOOM Eternal,https://www.3djuegos.com/32535/doom-eternal/,7,"[i5, r3]",8,"[1050, 1060, 1650]",50
108,Weird Hero,https://www.3djuegos.com/27728/weird-hero/,7,"[i5, r5]",4,[720],1


# Lógica

In [ ]:
[
    f"superioridad({marca}{n_s}, {marca}{n_i})"
    for marca in ["i", "r"]
    for n_s in range(9,2, -2)
    for n_i in range(3, 10, 2)
    if n_s > n_i
]
# for marca in ["i", "r"]:
#     for n_s in range(9,2, -2):
#         for n_i in range(3, 10, 2):
#             if n_s > n_i:
#                 print(f"superioridad({marca}{n_s}, {marca}{n_i})")


['superioridad(i9, i3)',
 'superioridad(i9, i5)',
 'superioridad(i9, i7)',
 'superioridad(i7, i3)',
 'superioridad(i7, i5)',
 'superioridad(i5, i3)',
 'superioridad(r9, r3)',
 'superioridad(r9, r5)',
 'superioridad(r9, r7)',
 'superioridad(r7, r3)',
 'superioridad(r7, r5)',
 'superioridad(r5, r3)']

In [ ]:
list(range(9, 2, -2))

[9, 7, 5, 3]

In [ ]:
def base_conocimiento(data):
    games_kb = pl.KnowledgeBase("games")
    facts = []

    # Equivalencia  
    tmp = [f"equivale(i{n},r{n})" for n in range(3,10,2)]
    tmp += [f"equivale({marca}{n},{marca}{n})" for marca in ["i", "r"] for n in range(3,10,2)]
    tmp += [f"equivale({marca}{n},{marca}{n})" for marca in ["r", "i"] for n in range(3,10,2)]

    facts += list(set(tmp))

    # Superioridad
    tmp = [
        f"superioridad({marca}{n_s},{marca}{n_i})"
        for marca in ["i", "r"]
        for n_s in range(9,2, -2)
        for n_i in range(3, 10, 2)
        if n_s > n_i
    ]

    facts += list(set(tmp))

    # Requisitos minimos
    for game in df.values:
        tmp = []
        for cpu in game[3]:
            for gpu in game[5]:
                tmp.append(f"requerimiento({game[0]},{game[2]},{cpu},{game[4]},{gpu},{game[6]})")
        facts += list(set(tmp))

    
    # Reglas
    facts.append("valSO(X,Y) :- X >= Y")
    facts.append("valRAM(X,Y) :- X >= Y")
    facts.append("valStorage(X,Y) :- X >= Y")
    facts.append("valGPU(X,Y) :- X >= Y")
    
    facts.append("superior(X,Y) :- equivale(Y,Z),superioridad(X,Z);superioridad(X,Z)")
    # facts.append("puede_jugar(A,B,C,X,Y,Z) :- requerimiento(A,B,C,X,Y,Z)")
    #facts.append("puede_jugar(A,B,C,X,Y,Z,B1,C1) :- requerimiento(A,B,C,X,Y,Z);valSO(B,B1),superior(C,C1)")
    # facts.append("puede_jugar(A,B,C,X,Y,Z,B1,C1,X1,Y1,Z1) :- requerimiento(A,B,C,X,Y,Z);valSO(B,B1), equivale(C,C1), valRAM(X,X1), valGPU(Y,Y1), valStorage(Z, Z1)")
    facts.append("puede_jugar(NAME,B,C,X,Y,Z) :- requerimiento(NAME,SO,CPU,RAM,GPU,STO)")
    facts.append("puede_jugar(SO,I_SO,CPU,I_CPU,RAM,I_RAM,GPU,I_GPU,STO,I_STO) :- valSO(SO,I_SO),equivale(CPU,I_CPU),valRAM(RAM,I_RAM),valGPU(GPU,I_GPU),valStorage(STO,I_STO)")
    facts.append("puede_jugar(SO,I_SO,CPU,I_CPU,RAM,I_RAM,GPU,I_GPU,STO,I_STO) :- valSO(SO,I_SO),superior(CPU,I_CPU),valRAM(RAM,I_RAM),valGPU(GPU,I_GPU),valStorage(STO,I_STO)")

    games_kb(facts)

    return facts, games_kb


In [ ]:
facts, games_kb = base_conocimiento(df)

In [ ]:
games_kb.query(pl.Expr("equivale(r9,r9)"))

['Yes']

In [ ]:
games_kb.query(pl.Expr("puede_jugar(Elden Ring,10,i5,12,1060,60)"))
games_kb.query(pl.Expr("puede_jugar(11,10,i9,i3,32,16,1650,1050,190,120)"))

['Yes']

In [ ]:
games_kb.query(pl.Expr("requerimiento(Elden Ring,B,C,D,E,F,E)"))

['No']

In [ ]:
games_kb.query(pl.Expr("valSO(5,5)"))

['Yes']

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=07e29549-6cbf-4e88-b69c-0fc48f2d6023' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>